In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from scipy.stats import spearmanr
import tifffile as tf
from false_positive_analysis import percent_false_positive
%config InlineBackend.figure_format='retina'

# Look at false positive rate

In [ ]:
#read in threshold mtx
#do you want to look at deepcell dots or daostar
deepcell = True
seed = 4

if deepcell == False:
    thresh_mtx = []
    for i in range(11):
        ch_mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/genebycell/daostar/seed_{seed}/Threshold_{i}/genebycell.csv", index_col=0)
        thresh_mtx.append(ch_mtx)

else:
    mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/genebycell/deepcell/seed_{seed}/genebycell.csv", index_col=0)

In [ ]:
#read in rnaseq data
rnaseq_1 = pd.read_csv("./RNAseq_files/nih3t3_FPKM.csv")
rnaseq_2 = pd.read_csv("./RNAseq_files/kallisto_NIH3T3.csv")
#rnaseq= rnaseq[["tracking_id","E14J B1"]]
rnaseq_1.columns = ["Genes","FPKM"]

In [ ]:
#codebook
codebook = pd.read_csv("/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/barcode_key/corrected_codebook.csv")
codebook.columns = ["Gene", "Brcd1", "Brcd2"]

In [ ]:
#separate into true and false codebook
fakebook = codebook[codebook["Gene"].str.startswith("fake")]
codebook = codebook.drop(fakebook.index)

In [ ]:
#calculate false positive
if deepcell == False:
    fp_list = []
    for i in range(11):
        fp, fake, norm_fpr= percent_false_positive(thresh_mtx[i], codebook, fakebook)
        percent_fp = fp["FP raw"].mean()
        mean_counts = fp["total_counts"].mean()
        sum_counts = fp["total_counts"].sum()
        fp_list.append([percent_fp,norm_fpr,mean_counts,sum_counts])
else:
    fp_list = []
    fp, fake, norm_fpr= percent_false_positive(mtx, codebook, fakebook)
    percent_fp = fp["FP raw"].mean()
    mean_counts = fp["total_counts"].mean()
    sum_counts = fp["total_counts"].sum()
    fp_list.append([percent_fp,norm_fpr,mean_counts,sum_counts])

#conver to df
df_stats = pd.DataFrame(fp_list)
df_stats.columns = ["percent fp","false positive rate","mean counts", "total sum"]
df_stats

In [ ]:
#plot fpr and percent falsepsotive
plt.plot(df_stats["percent fp"], label = "Percent False Positive")
plt.plot(df_stats["false positive rate"], label = "False Positive Rate")
plt.ylabel("Percent")
plt.xlabel("Thresholds tested")
sns.despine()
plt.legend()

In [ ]:
#plot average counts
plt.plot(df_stats["mean counts"])
plt.ylabel("Mean Total Counts")
plt.xlabel("Thresholds tested")
sns.despine()
plt.legend()

# Look at percent decoded dots

In [ ]:
def keep_dots_in_cells(mask, dot_locations):
    """
    A function to remove any dots outside of mask
    Parameter
    ---------
    mask = cellpose generated mask path
    dot_locations = dot_locations path
    """
    
    #read in data
    locations = pd.read_csv(dot_locations)
    #cellpose mask outputs (c,y,x)
    img = tf.imread(mask)
    #get x and y coordinates
    locations_xy = locations[["x","y"]].values.astype(int)
    dot_info = []
    #keep dots only in cells
    for i in range(len(locations)):
        x = locations_xy[i][0]
        y = locations_xy[i][1]
        if img[y,x] == 0:
            continue
        else:
            cell = img[y,x]
            dot_info.append([i,cell])
            
    dot_info = np.array(dot_info)
    
    #keep rows that have cells
    dots_in_cells = locations.loc[dot_info[:,0]]
    
    #add cell info
    dots_in_cells["cell number"] = dot_info[:,1]
    
    return dots_in_cells

In [ ]:
#look at total counts and keep only ones in masks

if deepcell == False:
    dot_list = []
    for i in range(11):
        mask = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/edges_deleted/MMStack_Pos0.ome.tif"
        dot_locations = f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_comb/channels_combined/Threshold_{i}/locations_z_0.csv"
        dots = keep_dots_in_cells(mask, dot_locations)
        dot_list.append(dots)
else:
    mask = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/edges_deleted/MMStack_Pos0.ome.tif"
    dot_locations = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos0/locations_z_0.csv"
    dots = keep_dots_in_cells(mask, dot_locations)
    dots = dots[dots["hyb"] != 12]

In [ ]:
#divide total counts by 4 barcoding rounds to get rough estimate on dots per barcode round
if deepcell == False:
    total_dots = []
    for _ in range(11):
        total_dots.append((len(dot_list[_])/4))
else:
    total_dots = len(dots)/4

total_dots

In [ ]:
#percent decoded plots
plt.plot(df_stats["total sum"].values / np.array(total_dots))
plt.ylabel("Percent Decoded")
plt.xlabel("Thresholds tested")
print(df_stats["total sum"].values / np.array(total_dots))

# RNA seq correlations

In [ ]:
#fp, fake, norm_fpr= percent_false_positive(thresh_mtx[6], codebook, fakebook)
fp, fake, norm_fpr= percent_false_positive(mtx, codebook, fakebook)

In [ ]:
#convert data to pseudobulk rnaseq data
if deepcell == False:
    bulk = pd.DataFrame(thresh_mtx[6].mean(axis=1)).reset_index()
    bulk.columns = ["Genes", "Counts"]
else:
    bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
    bulk.columns = ["Genes", "Counts"]

In [ ]:
#rename rnaseq columns and lowercase
rnaseq_2.columns = ["Genes", "TPM"]
rnaseq_1["Genes"]=rnaseq_1["Genes"].str.lower()
rnaseq_2["Genes"]=rnaseq_2["Genes"].str.lower()

In [ ]:
#merge
comb_1 = pd.merge(rnaseq_1,bulk)
comb_2 = pd.merge(rnaseq_2,bulk)

In [ ]:
#sort
sort_comb_1 = comb_1.sort_values(["FPKM"])
sort_comb_2 = comb_2.sort_values(["TPM"])

In [ ]:
#pearson calc old rnaseq
linreg = linregress(x = sort_comb_1["FPKM"], y = sort_comb_1["Counts"])
pearsonr = linreg[2]
pearsonr = round(pearsonr,2)
pearsonr

In [ ]:
#pearson calc new rnaseq
linreg = linregress(x = sort_comb_2["TPM"], y = sort_comb_2["Counts"])
pearsonr = linreg[2]
pearsonr = round(pearsonr,2)
pearsonr

In [ ]:
#plot correlation
plt.scatter(np.log10(sort_comb_2["Counts"]),np.log10(sort_comb_2["TPM"]), s = 5, alpha=0.5)
plt.ylabel("Bulk RNAseq Log10(FPKM)", fontsize=12)
plt.xlabel("Pseudobulk Log10(Counts)", fontsize=12)
plt.xticks(fontsize=12, rotation=0)
plt.yticks(fontsize=12, rotation=0)
plt.annotate(f"Pearson's r= {pearsonr}", (-1.5,1.5), fontsize=12)
sns.despine()

# Dots per pixel

In [ ]:
#import custom function
from neighbor_search_across import *
#import gen packages
import tifffile as tf
import pandas as pd

In [ ]:
#get mask src
mask = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/edges_deleted/MMStack_Pos0.ome.tif"
#get locations src
dot_locations = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos0/locations_z_0.csv"
#only keep dots in cells
dots = keep_dots_in_cells(mask, dot_locations)
#remove rehyb
dots = dots[dots["hyb"] != 12]
#sort by hyb
dots = dots.sort_values("hyb").reset_index(drop=True)
#remove random column
dots = dots.drop("Unnamed: 0", axis=1)

In [ ]:
#take a look
dots

In [ ]:
#calculate number of neighbors with expanding search radius
seeds= [0,1,2,3]
radii = np.linspace(1,6,20)

neighbor_list = []
for seed in seeds:
    for radius in radii:
        neighbors = neighbor_search(dots, hybs=12, num_barcodes=4, seed=seed, radius=radius)
        mean_neighbor = count_neighbors(neighbors)
        neighbor_list.append([seed,mean_neighbor,radius])

In [ ]:
#convert to df
neighbor_df = pd.DataFrame(neighbor_list)

In [ ]:
#take a look
neighbor_df

In [ ]:
import matplotlib.pyplot as plt
#plot the average number of neighbors across rounds using different seeds with various radii
for seed in neighbor_df[0].unique():
    plt.plot(neighbor_df[neighbor_df[0]==seed][2],neighbor_df[neighbor_df[0]==seed][1], 
             label = f"Seed {seed}", linewidth=0.5)
plt.legend()
plt.xlabel("Radius in Pixels")
plt.ylabel("Average number of neighbors")

# Check clusters within barcoding round

In [ ]:
#import custom function
from neighbor_search_within import *
#import ecdf plotting function
from ecdf import ecdf

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import tifffile as tf
import seaborn as sns
import pandas as pd

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=6
            ),
        name = "Round 1"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=6
                ),
            name = "Round 2"
            )
        )
    #-------------------------------------------
    
    fig.show()


In [ ]:
#get mask src
mask = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/edges_deleted/MMStack_Pos0.ome.tif"
#get dot src
dot_locations = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos0/locations_z_0.csv"
#keeps dots only in cells
dots = keep_dots_in_cells(mask, dot_locations)
#remove rehyb
locations = dots[dots["hyb"] != 12]

In [ ]:
#separate into barcoding rounds
barcoding_round = []

#separate locations by barcoding round
hyb_rounds = np.arange(0,12,1)
temp = []
for h in hyb_rounds:
    if h == hyb_rounds[len(hyb_rounds)-1]:
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)
        comp_round = pd.concat(temp)
        barcoding_round.append(comp_round) 
    elif (h % (12/4) != 0) or (h == 0):
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)
    else:
        comp_round = pd.concat(temp)
        barcoding_round.append(comp_round)
        temp = []
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)

#remove temp list
del temp

In [ ]:
#read in img
img = tf.imread("../aberration_corrected/HybCycle_0/MMStack_Pos0.ome.tif")

In [ ]:
#plot dots on top of image
plot_2d_locs_on_2d_image(barcoding_round[0], df_locs_2d_2=None, img_2d=img[4],add_trace = False, zmax=1000)

In [ ]:
#perform various radius searches and calculate dot density
neigh_list = []
index_list = []
radius_list = [1,2,3,4,5]
for radius in radius_list:
    neighbors,seed = neighbor_search_within(locations, hybs=12, num_barcodes=4, seed=0, radius=radius)
    density,index = density_estimate(neighbors, radius=radius, average=False)
    index_list.append(index)
    neigh_list.append(density)

In [ ]:
#combine dfs
comb = pd.concat(neigh_list).reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
#plot distributions
ecdf.plot(comb, label_column = "radius", val_column = "number of neighbors/pixel", 
          conf = False, color = ["red","blue","green","purple","orange"])
plt.xlabel("# of neighbors/pixel")
plt.ylabel("ECDF")
plt.show()

In [ ]:
#perform various radius searches and get average density
neigh_list = []
radius_list = np.linspace(1,5,20)
for radius in radius_list:
    neighbors,seed = neighbor_search_within(locations, hybs=12, num_barcodes=4, seed=0, radius=radius)
    density,index = density_estimate(neighbors, radius=radius, average=True)
    neigh_list.append(density)

In [ ]:
#plot average density with expaning search radius
plt.plot(np.linspace(1,5,20), neigh_list)
sns.despine()
plt.xlabel("Search Radius in Pixels")
plt.ylabel("Average Neighbors/Pixel")

In [ ]:
#get mask
mask = "/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/edges_deleted/MMStack_Pos0.ome.tif"
#calculate area of mask
area_mask = np.sum(tf.imread(mask)>=1)

In [ ]:
len(barcoding_round[0])/area_mask